# Test aggregation functions

In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| output: false
import numpy as np
import tempfile
from pandas import DataFrame
from pathlib import Path
import pandas as pd

from spannerlib.span import Span
from spannerlib.session import Session,test_session

In [ ]:
def lexic_concat(strings):
    strings = list(strings)
    return " ".join(sorted(strings))

test_session(
    [
        """
        new Text(str)
        Text("he")
        Text("llo")
        Text("ho")

        Prompt(concat(P))<-Text(P)
        """,
    
        """
        ?Prompt(P)
        """
    ],
    [
        None,
        DataFrame({'P': ['he ho llo']})
    ],
    agg_funcs=[
        ['concat',lexic_concat,[str],[str]],
    ],
)


'?Prompt(P)'

,P
0,he ho llo


In [ ]:
test_session(
    [
        """
        new nums(int,int,int)
        nums(1,2,3)
        nums(1,5,6)
        nums(2,3,4)
        nums(2,8,9)
        
        aggregations(X,sum(Y),min(Z)) <- nums(X,Y,Z)
        ?aggregations(A,B,C)
        """,
    
    ],
    [
            DataFrame([
                [1, 7, 3],
                [2, 11, 4]]
                , columns=['A','B','C'])
    ],
    agg_funcs=[
        ['concat',lexic_concat,[str],[str]],
    ],
)

'?aggregations(A,B,C)'

,A,B,C
0,1,7,3
1,2,11,4


In [ ]:
# TODO FROM here, debug these and then make tests with multiple aggs on the same vars,
test_session(
    [
        """
        new nums(int,int,int)
        nums(1,2,3)
        nums(1,5,6)
        nums(2,3,4)
        nums(2,8,9)
        
        aggregations(X,sum(Y),min(X)) <- nums(X,Y,Z)
        ?aggregations(A,B,C)
        """,
    
    ],
    [
            DataFrame([
                [1, 7, 1],
                [2, 11, 2]]
                , columns=['A','B','C'])
    ],
    agg_funcs=[
        ['concat',lexic_concat,[str],[str]],
    ],
)

Exception: During execution of statement 
"?aggregations(A,B,C)
"the following exception was raised:
'DataFrame' object has no attribute 'to_frame'
